In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
train=pd.read_csv('Dataset.csv')

In [4]:
train=train.rename(columns={
    'Unnamed: 0':'idx', 
   
    'NUM_Comments':'Num_Comments',
    
    'Coronavirus':'Label1Coronovirus', 
    'Politics':'Label2Politics', 
    'Non-Political':'Label3Non-Political',
    'Unnamed: 11':'Label4None', 
    '| Repost |':'Label5Repost', 
    'Unverified':'Label6Unverified', 
    'Policy/Economy':'Label7Policy/Economy',
    'Photography':'Label8Photography', 
    'Business/Finance':'Label9Buisness/Finance', 
    'AskIndia':'Label10AskIndia',
    'Scheduled':'Label11Scheduled',
    '| Stickied Topic |':'Label12Stickied-Topic', 
    '| Unverified Content / Disreputed Source |':'Label13Unverified-Content',
    'Science/Technology':'Label14Science/Technology', 
    'Food':'Label15Food', 
    '| Not Original/Relevant Title |':'Label16Not Original',
    '| Social Media Rules |':'Label17Social Media', 
    'CAA-NRC-NPR':'Label18CAA/NRC', 
    'Megathread':'Label19Megathread', 
    '🥳':'Label20Unk'})

In [11]:
print(f"Label1Ctrain{train['Label1Coronovirus'].value_counts()[1]}")
print(f"Label2Ptrain{train['Label2Politics'].value_counts()[1]}")
print(f"Label3Ntrain{train['Label3Non-Political'].value_counts()[1]}")
      
print(f"Label4Ntrain{train['Label4None'].value_counts()[1]}")
print(f"Label5Rtrain{train['Label5Repost'].value_counts()[1]}")
print(f"Label6Utrain{train['Label6Unverified'].value_counts()[1]}")

#print(f"Label7Ptrain{train['Label7Policy/Economy'].value_counts()[1]}")
#print(f"Label8Ptrain{train['Label8Photography'].value_counts()[1]}")
#print(f"Label9Btrain{train['Label9Buisness/Finance'].value_counts()[1]}")

print(f"Label10train{train['Label10AskIndia'].value_counts()[1]}")

print(f"Label11train{train['Label11Scheduled'].value_counts()[1]}")
#print(f"Label12train{train['Label12Stickied-Topic'].value_counts()[1]}")
#print(f"Label13train{train['Label13Unverified-Content'].value_counts()[1]}")
#print(f"Label14train{train['Label14Science/Technology'].value_counts()[1]}")
#print(f"Label15train{train['Label15Food'].value_counts()[1]}")
#print(f"Label16train{train['Label16Not Original'].value_counts()[1]}")
#print(f"Label17train{train['Label17Social Media'].value_counts()[1]}")
#print(f"Label18train{train['Label18CAA/NRC'].value_counts()[1]}")
#print(f"Label19train{train['Label19Megathread'].value_counts()[1]}")
#print(f"Label20train{train['Label20Unk'].value_counts()[1]}")

Label1Ctrain306
Label2Ptrain111
Label3Ntrain94
Label4Ntrain6
Label5Rtrain5
Label6Utrain4
Label7Ptrain18
Label8Ptrain11
Label9Btrain15
Label10train82
Label11train7
Label12train1
Label13train1
Label14train13
Label15train9
Label16train3
Label17train1
Label18train3
Label19train1
Label20train1


In [15]:
dropping_cols=['Label20Unk','Label19Megathread','Label18CAA/NRC','Label17Social Media',
              'Label16Not Original','Label13Unverified-Content','Label12Stickied-Topic',
              ]

In [16]:
other=['Label7Policy/Economy',
      'Label8Photography',
      'Label9Buisness/Finance',
       'Label14Science/Technology',
       'Label15Food',
       ]

In [88]:
test_set=['Label4None',
         'Label5Repost',
         'Label6Unverified',
         'Label11Scheduled']

In [18]:
train=train.drop(dropping_cols,axis=1)

In [20]:
train[train['Label1Coronovirus']==1].shape

(306, 21)

In [19]:
train.shape

(692, 21)

In [59]:
train['Label4Objects']=0

In [38]:
for label in other:
    for row in train

In [47]:
for row in train.index:
    for col in other:
        if(train[col][row]):
            print(train[col][row])
            break

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [63]:
for i in range(len(train)):
    print(train.loc[i,'Title'])
    train.loc[i,'Label4Objects']=1
    break

Coronavirus (COVID-19) Megathread - News and Updates - 3


In [69]:
for i in range(len(train)):
    for nam in other:
        if (train.loc[i,nam]==1):
            train.loc[i,'Label4Objects']=1
    
    

In [72]:
train[train['Label4Objects']==1]

,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,...,Label5Repost,Label6Unverified,Label7Policy/Economy,Label8Photography,Label9Buisness/Finance,Label10AskIndia,Label11Scheduled,Label14Science/Technology,Label15Food,Label4Objects
23,23,Covid-19 lockdown impact: Unemployment rate ri...,28,fweqm5,https://www.livemint.com/news/india/covid-19-l...,1,1.586266e+09,NaN,0,0,...,0,0,1,0,0,0,0,0,0,1
27,27,Starry timelapse from my roof. 600 photos{f/3....,33,fwctr4,https://v.redd.it/a3y5judnabr41,2,1.586258e+09,NaN,0,0,...,0,0,0,1,0,0,0,0,0,1
31,31,Isn't it just fucking great how companies like...,242,fw14l3,https://www.reddit.com/r/india/comments/fw14l3...,52,1.586217e+09,And I know that a lot of upper management in t...,0,0,...,0,0,0,0,1,0,0,0,0,1
44,44,India’s richest man lost $300 million a day in...,12,fwfkf8,https://qz.com/india/1833510/coronavirus-wipes...,5,1.586270e+09,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1
70,70,How the 2020 Stock Market Crash is Different f...,6,fwf01u,https://www.financereads.in/how-the-2020-stock...,0,1.586267e+09,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1
76,76,"Dudhsagar Falls, Goa Karnataka Border, India [...",5,fwfy3q,https://i.redd.it/xiav3rxyhcr41.jpg,0,1.586272e+09,NaN,0,0,...,0,0,0,1,0,0,0,0,0,1
79,79,Vodafone is having a network outage,6,fwfp4u,https://www.reddit.com/r/india/comments/fwfp4u...,1,1.586271e+09,Just one look at the twitter and it's showing ...,0,0,...,0,0,0,0,0,0,0,1,0,1
80,80,17 new coronavirus cases reported in Jammu and...,10,fwc8kv,https://www.livemint.com/news/india/17-new-cor...,0,1.586255e+09,NaN,0,0,...,0,0,1,0,0,0,0,0,0,1
90,90,"As IIT, IIM students lose job offers, institut...",58,fw1qin,https://www.moneycontrol.com/news/business/eco...,21,1.586219e+09,NaN,0,0,...,0,0,1,0,0,0,0,0,0,1
101,101,Indian food after Heart surgery/Angioplasty?,4,fwen8i,https://afitindian.com/tips-for-a-healthy-indi...,0,1.586265e+09,NaN,0,0,...,0,0,0,0,0,0,0,0,1,1


In [74]:
train=train.drop(other,axis=1)

In [75]:
train.head()

,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label4None,Label5Repost,Label6Unverified,Label10AskIndia,Label11Scheduled,Label4Objects
0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0,0,0,0,0
1,1,Firozabad police fact-checking zee news (trans...,680,fwcz7h,https://i.redd.it/jbal4gxocbr41.jpg,62,1.586258e+09,NaN,0,1,0,0,0,0,0,0,0
2,2,Donald Trump talks of ‘retaliation’ if India t...,371,fwclvi,https://economictimes.indiatimes.com/industry/...,135,1.586257e+09,NaN,1,0,0,0,0,0,0,0,0
3,3,Mom came up with this impressive idea to celeb...,3129,fvz69t,https://i.redd.it/gurfdfd1e7r41.jpg,183,1.586210e+09,NaN,0,0,1,0,0,0,0,0,0
4,4,9 PM 9 Minutes: House Catches Fire In Bihar; G...,181,fwdysc,https://news.abplive.com/news/india/coronaviru...,14,1.586262e+09,NaN,0,0,1,0,0,0,0,0,0


In [89]:
test_set

['Label4None', 'Label5Repost', 'Label6Unverified', 'Label11Scheduled']

In [84]:
test_dat=train[train['Label4None']==1]

In [85]:
test_dat=test_dat.append(train[train['Label5Repost']==1])
test_dat=test_dat.append(train[train['Label6Unverified']==1])
test_dat=test_dat.append(train[train['Label11Scheduled']==1])


In [86]:
test_dat.shape

(22, 17)

In [90]:
train=train.drop(test_set,axis=1)

In [91]:
train

,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label10AskIndia,Label4Objects
0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0
1,1,Firozabad police fact-checking zee news (trans...,680,fwcz7h,https://i.redd.it/jbal4gxocbr41.jpg,62,1.586258e+09,NaN,0,1,0,0,0
2,2,Donald Trump talks of ‘retaliation’ if India t...,371,fwclvi,https://economictimes.indiatimes.com/industry/...,135,1.586257e+09,NaN,1,0,0,0,0
3,3,Mom came up with this impressive idea to celeb...,3129,fvz69t,https://i.redd.it/gurfdfd1e7r41.jpg,183,1.586210e+09,NaN,0,0,1,0,0
4,4,9 PM 9 Minutes: House Catches Fire In Bihar; G...,181,fwdysc,https://news.abplive.com/news/india/coronaviru...,14,1.586262e+09,NaN,0,0,1,0,0
5,5,Cartoon by Pencilashan,120,fwep0p,https://i.redd.it/6g8nk0t5zbr41.jpg,7,1.586266e+09,NaN,0,1,0,0,0
6,6,Pandemic impact: Mukesh Ambani's net worth dro...,138,fwd6mm,https://economictimes.indiatimes.com/markets/s...,21,1.586259e+09,NaN,0,0,1,0,0
7,7,Symbiotic Relationship,1008,fw163b,https://i.redd.it/l6ad1563z7r41.jpg,62,1.586217e+09,NaN,0,1,0,0,0
8,8,Coronavirus: How Shashi Tharoor Spent His MP F...,116,fwcvid,https://www.ndtv.com/kerala-news/coronavirus-h...,12,1.586258e+09,NaN,1,0,0,0,0
9,9,Mainstream Indian media for you. Palestine is ...,355,fw69nv,https://i.redd.it/v3ygizrab9r41.jpg,44,1.586233e+09,NaN,0,0,0,0,0


In [92]:
train['Label5AskIndia']=train['Label10AskIndia']

In [93]:
train

,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label10AskIndia,Label4Objects,Label5AskIndia
0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0,0
1,1,Firozabad police fact-checking zee news (trans...,680,fwcz7h,https://i.redd.it/jbal4gxocbr41.jpg,62,1.586258e+09,NaN,0,1,0,0,0,0
2,2,Donald Trump talks of ‘retaliation’ if India t...,371,fwclvi,https://economictimes.indiatimes.com/industry/...,135,1.586257e+09,NaN,1,0,0,0,0,0
3,3,Mom came up with this impressive idea to celeb...,3129,fvz69t,https://i.redd.it/gurfdfd1e7r41.jpg,183,1.586210e+09,NaN,0,0,1,0,0,0
4,4,9 PM 9 Minutes: House Catches Fire In Bihar; G...,181,fwdysc,https://news.abplive.com/news/india/coronaviru...,14,1.586262e+09,NaN,0,0,1,0,0,0
5,5,Cartoon by Pencilashan,120,fwep0p,https://i.redd.it/6g8nk0t5zbr41.jpg,7,1.586266e+09,NaN,0,1,0,0,0,0
6,6,Pandemic impact: Mukesh Ambani's net worth dro...,138,fwd6mm,https://economictimes.indiatimes.com/markets/s...,21,1.586259e+09,NaN,0,0,1,0,0,0
7,7,Symbiotic Relationship,1008,fw163b,https://i.redd.it/l6ad1563z7r41.jpg,62,1.586217e+09,NaN,0,1,0,0,0,0
8,8,Coronavirus: How Shashi Tharoor Spent His MP F...,116,fwcvid,https://www.ndtv.com/kerala-news/coronavirus-h...,12,1.586258e+09,NaN,1,0,0,0,0,0
9,9,Mainstream Indian media for you. Palestine is ...,355,fw69nv,https://i.redd.it/v3ygizrab9r41.jpg,44,1.586233e+09,NaN,0,0,0,0,0,0


In [94]:
train=train.drop('Label10AskIndia',axis=1)

In [95]:
train

,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label4Objects,Label5AskIndia
0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0
1,1,Firozabad police fact-checking zee news (trans...,680,fwcz7h,https://i.redd.it/jbal4gxocbr41.jpg,62,1.586258e+09,NaN,0,1,0,0,0
2,2,Donald Trump talks of ‘retaliation’ if India t...,371,fwclvi,https://economictimes.indiatimes.com/industry/...,135,1.586257e+09,NaN,1,0,0,0,0
3,3,Mom came up with this impressive idea to celeb...,3129,fvz69t,https://i.redd.it/gurfdfd1e7r41.jpg,183,1.586210e+09,NaN,0,0,1,0,0
4,4,9 PM 9 Minutes: House Catches Fire In Bihar; G...,181,fwdysc,https://news.abplive.com/news/india/coronaviru...,14,1.586262e+09,NaN,0,0,1,0,0
5,5,Cartoon by Pencilashan,120,fwep0p,https://i.redd.it/6g8nk0t5zbr41.jpg,7,1.586266e+09,NaN,0,1,0,0,0
6,6,Pandemic impact: Mukesh Ambani's net worth dro...,138,fwd6mm,https://economictimes.indiatimes.com/markets/s...,21,1.586259e+09,NaN,0,0,1,0,0
7,7,Symbiotic Relationship,1008,fw163b,https://i.redd.it/l6ad1563z7r41.jpg,62,1.586217e+09,NaN,0,1,0,0,0
8,8,Coronavirus: How Shashi Tharoor Spent His MP F...,116,fwcvid,https://www.ndtv.com/kerala-news/coronavirus-h...,12,1.586258e+09,NaN,1,0,0,0,0
9,9,Mainstream Indian media for you. Palestine is ...,355,fw69nv,https://i.redd.it/v3ygizrab9r41.jpg,44,1.586233e+09,NaN,0,0,0,0,0


In [98]:
print(train['Label1Coronovirus'].value_counts())
print(train['Label2Politics'].value_counts())
print(train['Label3Non-Political'].value_counts())
print(train['Label4Objects'].value_counts())
print(train['Label5AskIndia'].value_counts())


0    386
1    306
Name: Label1Coronovirus, dtype: int64
0    581
1    111
Name: Label2Politics, dtype: int64
0    598
1     94
Name: Label3Non-Political, dtype: int64
0    626
1     66
Name: Label4Objects, dtype: int64
0    610
1     82
Name: Label5AskIndia, dtype: int64


In [99]:
train.shape

(692, 13)

In [100]:
train2=train.copy()

In [102]:
train2.shape

(692, 13)

In [106]:
train2=train2.drop(train2[train2['Label1Coronovirus']==1][:1])

KeyError: "['idx' 'Title' 'Score' 'ID' 'Image' 'Num_Comments' 'Created' 'Body'\n 'Label1Coronovirus' 'Label2Politics' 'Label3Non-Political'\n 'Label4Objects' 'Label5AskIndia'] not found in axis"

In [117]:
dropping_list=train2[train2['Label1Coronovirus']==1].index[-186:]

In [119]:
train=train.drop(dropping_list)

In [120]:
train.shape

(506, 13)

In [121]:
print(train['Label1Coronovirus'].value_counts())
print(train['Label2Politics'].value_counts())
print(train['Label3Non-Political'].value_counts())
print(train['Label4Objects'].value_counts())
print(train['Label5AskIndia'].value_counts())

0    386
1    120
Name: Label1Coronovirus, dtype: int64
0    395
1    111
Name: Label2Politics, dtype: int64
0    412
1     94
Name: Label3Non-Political, dtype: int64
0    440
1     66
Name: Label4Objects, dtype: int64
0    424
1     82
Name: Label5AskIndia, dtype: int64


In [122]:
train.to_csv('updated_train.csv')

In [123]:
test_dat.to_csv('Test_data.csv')

In [124]:
train.shape

(506, 13)

In [129]:
train=pd.read_csv('updated_train.csv')

In [137]:
png=0
jpg=0
for i in range(506):
    a=train.loc[i,'Image']
    print(f"{i} ,{a[-4:]}")
    if a[-4:]=='.png':
        png+=1
        print("HEllo")
    if a[-4:]=='.jpg':
        jpg+=1
        print("Kello")


0 ,s_3/
1 ,.jpg
Kello
2 ,.cms
3 ,.jpg
Kello
4 ,0840
5 ,.jpg
Kello
6 ,.cms
7 ,.jpg
Kello
8 ,7170
9 ,.jpg
Kello
10 ,html
11 ,html
12 ,ims/
13 ,ases
14 ,ITh4
15 ,ply/
16 ,dra/
17 ,html
18 ,G8YI
19 ,0173
20 ,6619
21 ,.cms
22 ,.jpg
Kello
23 ,true
24 ,7119
25 ,5860
26 ,orts
27 ,br41
28 ,true
29 ,true
30 ,amp/
31 ,ola/
32 ,963/
33 ,_to/
34 ,1977
35 ,.cms
36 ,ues/
37 ,.cms
38 ,7105
39 ,ter/
40 ,guBL
41 ,LzmU
42 ,7750
43 ,html
44 ,ahoo
45 ,eaks
46 ,html
47 ,ckup
48 ,4868
49 ,ama/
50 ,get/
51 ,_my/
52 ,4046
53 ,_at/
54 ,ter/
55 ,html
56 ,html
57 ,6564
58 ,ood/
59 ,html
60 ,.jpg
Kello
61 ,html
62 ,ight
63 ,.ece
64 ,use/
65 ,4-06
66 ,4837
67 ,907/
68 ,-27/
69 ,html
70 ,008/
71 ,orts
72 ,tans
73 ,nel/
74 ,s=09
75 ,s=20
76 ,.jpg
Kello
77 ,_an/
78 ,.ece
79 ,age/
80 ,html
81 ,.cms
82 ,4-06
83 ,7314
84 ,html
85 ,708/
86 ,html
87 ,zled
88 ,html
89 ,.in/
90 ,html
91 ,html
92 ,.cms
93 ,1247
94 ,558/
95 ,4896
96 ,.ece
97 ,ery/
98 ,.ece
99 ,ndia
100 ,3324
101 ,sty/
102 ,ne )
103 ,maat
104 ,ter/
105 ,ate/
10

In [133]:
png,jpg

(8, 30)

In [134]:
train.head()

,Unnamed: 0,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label4Objects,Label5AskIndia
0,0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0
1,1,1,Firozabad police fact-checking zee news (trans...,680,fwcz7h,https://i.redd.it/jbal4gxocbr41.jpg,62,1.586258e+09,NaN,0,1,0,0,0
2,2,2,Donald Trump talks of ‘retaliation’ if India t...,371,fwclvi,https://economictimes.indiatimes.com/industry/...,135,1.586257e+09,NaN,1,0,0,0,0
3,3,3,Mom came up with this impressive idea to celeb...,3129,fvz69t,https://i.redd.it/gurfdfd1e7r41.jpg,183,1.586210e+09,NaN,0,0,1,0,0
4,4,4,9 PM 9 Minutes: House Catches Fire In Bihar; G...,181,fwdysc,https://news.abplive.com/news/india/coronaviru...,14,1.586262e+09,NaN,0,0,1,0,0


In [139]:
train[train['Body'].notnull()]

,Unnamed: 0,idx,Title,Score,ID,Image,Num_Comments,Created,Body,Label1Coronovirus,Label2Politics,Label3Non-Political,Label4Objects,Label5AskIndia
0,0,0,Coronavirus (COVID-19) Megathread - News and U...,287,fqqdsg,https://www.reddit.com/r/india/comments/fqqdsg...,6834,1.585451e+09,**Central thread for sharing coronavirus News ...,1,0,0,0,0
16,16,16,Kamal Haasan Writes To Prime Minister Narendra...,55,fwemlb,https://www.reddit.com/r/india/comments/fwemlb...,2,1.586265e+09,"**Many Indians didn't question the lockdown, t...",0,0,0,0,0
31,31,31,Isn't it just fucking great how companies like...,242,fw14l3,https://www.reddit.com/r/india/comments/fw14l3...,52,1.586217e+09,And I know that a lot of upper management in t...,0,0,0,1,0
33,33,33,What do you do when your own family subscribes...,26,fwdgbn,https://www.reddit.com/r/india/comments/fwdgbn...,23,1.586260e+09,I hate the Godi media so much right now. Durin...,0,1,0,0,0
36,36,36,Marriage and social issues,36,fwbgle,https://www.reddit.com/r/india/comments/fwbgle...,11,1.586252e+09,I cant be the only one who grew up in a very t...,0,0,0,0,1
39,39,39,Is India Getting Better?,32,fwc5vu,https://www.reddit.com/r/india/comments/fwc5vu...,8,1.586255e+09,Hello everyone! Lot of things going down these...,0,0,1,0,0
49,49,49,I am an exmuslim. AMA!,123,fw2u16,https://www.reddit.com/r/india/comments/fw2u16...,201,1.586223e+09,Hi! \n\n\nI am an in-the-closet exmuslim who...,0,0,1,0,0
50,50,50,Men who are 30+ and have decided not to get ma...,248,fvy95j,https://www.reddit.com/r/india/comments/fvy95j...,200,1.586207e+09,The corona virus has given me some time to thi...,0,0,0,0,1
51,51,51,On losing hope and daydreaming about ceasing m...,46,fw88wr,https://www.reddit.com/r/india/comments/fw88wr...,18,1.586240e+09,I am 29 year old guy who happens to be gay. I ...,0,0,1,0,0
53,53,53,"Random Daily Discussion Thread - April 07, 202...",15,fwdeu4,https://www.reddit.com/r/india/comments/fwdeu4...,843,1.586260e+09,"^Beep ^Boop ^Bot, ^I ^am ^a ^bot! ^if ^any ^pr...",0,0,0,0,0


In [150]:
train2[train2['Label1Coronovirus']==0 and train2['Label2Politics']==0]and train2['Label3Non-Political']==0 and train2['Label4Objects']==0
      and train2['Label5AskIndia']==0)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [157]:
train2['Sum']=train2['Label1Coronovirus']+train2['Label2Politics']+train2['Label3Non-Political']+train2['Label4Objects']+train2['Label5AskIndia']

In [160]:
output_less=train2[train2['Sum']!=1].index

In [162]:
train2.drop(output_less,inplace=True)

In [164]:
train2['Sum'].sum()

659

In [165]:
train2.shape

(659, 14)

In [166]:
train['Sum']=train['Label1Coronovirus']+train['Label2Politics']+train['Label3Non-Political']+train['Label4Objects']+train['Label5AskIndia']

In [167]:
output_less=train[train['Sum']!=1].index

In [168]:
train.drop(output_less,inplace=True)

In [170]:
train.shape, train['Sum'].sum()

((473, 15), 473)

In [171]:
del train['Sum']

In [172]:
train.to_csv('updated_train.csv')

In [ ]:
train

In [ ]:
import urllib.request 
from PIL import Image
url='https://i.redd.it/jbal4gxocbr41.jpg'

image=Image.open(urllib.request.urlopen(url))
width,height=image.size

print(width,height)
#print(image.shape)
plt.imshow(image)
k=np.array(image)